In [ ]:
import time

# Start time of script execution 
ExecuteStart = system.date.now()
print "Execute Script Start Time: ", ExecuteStart

# Define start and end dates
sDate = system.date.getDate(2024, 1, 11)
eDate = system.date.getDate(2024, 1, 12)

# Format dates to a readable format
startDate = system.date.format(sDate, "yyyy-MM-dd HH:mm:ss")
endDate = system.date.format(eDate, "yyyy-MM-dd HH:mm:ss")

# Calculate the number of days between the dates
numDays = system.date.daysBetween(sDate, eDate)
timeout = numDays * 1000 * 24 * 12  # Set timeout based on days

# Set return size for 1-minute intervals
returnSize = numDays * 24 * 60   # 1440 intervals (1-minute) per day

# Print debug info
print "Start Date: ", startDate
print "End Date: ", endDate
print "Number of Days: ", numDays
print "Script Timeout: ", timeout
print "Return Size: ", returnSize

# Define paths to query
paths = [
    "[hist/iqaluit scada system - wtp:wtp]analog/fit_5001/val",
]

# Define aggregation modes
aggregationModes = [
    "Average",  # Change to other modes if needed
]

# Query historical data
data = system.tag.queryTagHistory(
    paths=paths,
    startDate=startDate,
    endDate=endDate,
    returnSize=returnSize,
    aggregationModes=aggregationModes,
    returnFormat='Tall',
    timeout=timeout
)

# Extract column names as a proper list of strings
columnNames = list(data.getColumnNames())
print "Column Names: ", columnNames

# Convert the dataset to a Python dataset for manipulation
pyDataSet = system.dataset.toPyDataSet(data)
# Check the number of rows in the dataset
rowCount = pyDataSet.getRowCount()
print "Number of rows in dataset:", rowCount

# Shared counter for completed inserts (using system.util to track across threads)
if 'completed_inserts' not in system.util.getGlobals():
    system.util.getGlobals()['completed_inserts'] = 0

def increment_completed_inserts():
    # Increment the counter in a thread-safe manner
    current_value = system.util.getGlobals()['completed_inserts']
    system.util.getGlobals()['completed_inserts'] = current_value + 1

# Function to insert rows asynchronously with retry logic
def insertRowAsync(row, attempt=1, max_attempts=3):
    try:
        tag_name = row[0]  # First column: tag path (or tag name)
        value = row[1]     # Second column: value
        T_stamp = row[3]   # 4th column: timestamp

        # Round the value to 2 decimal places
        value = round(value, 2) if isinstance(value, (float, int)) else value

        # Avoid inserting NULL values
        if value is not None:
            # Prepare query for individual insert
            query = """
                INSERT INTO records (T_stamp, tag_name, value)
                VALUES (?, ?, ?)
                ON CONFLICT (T_stamp, tag_name)
                DO UPDATE SET value = EXCLUDED.value;
            """
            # Insert the row into the database
            system.db.runPrepUpdate(query, [T_stamp, tag_name, value], "IQ_Report")
            print "Row inserted with T_stamp:", T_stamp
        else:
            print "Skipping row due to None value:", row

        # Increment the completed counter when successful
        increment_completed_inserts()

    except Exception:
        # Retry logic: Retry up to max_attempts times
        if attempt < max_attempts:
            print "Error inserting row with T_stamp:", T_stamp, "| Attempt:", attempt, "| Retrying..."
            system.util.invokeLater(lambda: insertRowAsync(row, attempt + 1, max_attempts), 1000)  # Retry after 1 second
        else:
            print "Failed to insert row with T_stamp after", max_attempts, "attempts:", T_stamp
            increment_completed_inserts()  # Count it as completed after max retries

# Increased sleep time (200ms) to avoid overwhelming the database
sleep_time = 200  # 200 milliseconds between each insert

# Loop through the dataset and insert each row asynchronously with retry logic and delay
for idx, row in enumerate(pyDataSet):
    system.util.invokeLater(lambda r=row: system.util.invokeAsynchronous(lambda: insertRowAsync(r)), idx * sleep_time)

# Wait for all inserts to finish before ending the script
def wait_for_inserts():
    print "Waiting for all inserts to complete..."
    while system.util.getGlobals()['completed_inserts'] < rowCount:
        print "Completed inserts:", system.util.getGlobals()['completed_inserts'], "out of", rowCount
        time.sleep(1)  # Sleep for 1 second before checking again
        
    print "All inserts completed."

wait_for_inserts()

# End time of script execution
ExecuteEnd = system.date.now()
print "Execute Script End Time: ", ExecuteEnd
print "It took", system.date.secondsBetween(ExecuteStart, ExecuteEnd), "seconds to execute"
